# Problem 1 

## (a)

\begin{align*}
dC(r_t,t) &= \frac{\partial C}{\partial r_t}(r_t,t) dr_t + \frac{\partial C}{\partial t}(r_t,t) dt + \frac{1}{2} \frac{\partial^2 C}{\partial r_t^2}(r_t,t) \langle dr_t, dr_t \rangle \\
dC(r_t,t) &= \frac{\partial C}{\partial r_t}(r_t,t) dr_t + \frac{\partial C}{\partial t}(r_t,t) dt + \frac{1}{2} \frac{\partial^2 C}{\partial r_t^2}(r_t,t) \beta^2(r_t,t) dt \\
\Rightarrow & \frac{\partial C}{\partial r_t}(r_t,t) \alpha(r_t, t) dt + \frac{\partial C}{\partial t}(r_t,t) dt + \frac{1}{2} \frac{\partial^2 C}{\partial r_t^2}(r_t,t) \beta^2(r_t,t) dt = rC \\
\Rightarrow & \left(\frac{\partial C}{\partial r_t} \alpha(r_t, t) + \frac{\partial C}{\partial t} + \frac{1}{2} \frac{\partial^2 C}{\partial r_t^2} \beta^2(r_t,t)\right) dt = rC dt
\end{align*}


## (b)

In [1]:
import numpy as np

In [2]:
class Vasicek:

    def __init__(self,kappa,theta,sigma):
        self.kappa=kappa
        self.theta=theta
        self.sigma=sigma

In [3]:
hw3dynamics = Vasicek(kappa=3,theta=0.05,sigma=0.03)

In [4]:
class Bond:

    def __init__(self, T):
        self.T=T


In [5]:
hw3contract = Bond(T=5)

In [6]:
class FDexplicitEngine:

    def __init__(self, rMax, rMin, deltar, deltat, useUpwind):
        self.rMax=rMax
        self.rMin=rMin
        self.deltar=deltar
        self.deltat=deltat
        self.useUpwind=useUpwind

    def price_bond_vasicek(self,contract,dynamics):


        T = contract.T
        N=round(T/self.deltat)
        if abs(N-T/self.deltat) > 1e-12:
            raise ValueError("Bad delta t")

        r=np.arange(self.rMax,self.rMin-self.deltar/2,-self.deltar)  
        bondprice=np.ones(np.size(r))

        if self.useUpwind:
            nu= dynamics.kappa*(dynamics.theta - r)
            p1 = (np.power(dynamics.sigma, 2) * self.deltat) / (2 * np.power(self.deltar, 2))
            p2 = self.deltat / self.deltar * nu
            indicator = np.where(r < dynamics.theta, 1, 0)
            qu=  p1 + p2 * indicator    
            qd= p1 + p2 * (indicator-1)    
            qm= 1-qu-qd  
        else:
            # drt = kappa(theta - rt)dt + sigmadWt 
            nu = dynamics.kappa*(dynamics.theta - r)
            qu= (dynamics.sigma**2*self.deltat/(self.deltar **2) + nu*self.deltat/self.deltar)/2    
            qd= (dynamics.sigma**2*self.deltat/(self.deltar **2) - nu*self.deltat/self.deltar)/2   
            qm= 1-qu-qd  

        for t in np.arange(N-1,-1,-1)*self.deltat:
            bondprice=1/(1+r*self.deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))


            bondprice[0]=2*bondprice[1]-bondprice[2]
            bondprice[-1]=2*bondprice[-2]-bondprice[-3]

        return (r, bondprice)


In [7]:
hw3FD = FDexplicitEngine(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=False)

In [8]:
(r, bondprice) = hw3FD.price_bond_vasicek(hw3contract,hw3dynamics)

In [9]:
np.set_printoptions(precision=4,suppress=True)
displayrows=(r<0.15+hw3FD.deltar/2) & (r>0.0-hw3FD.deltar/2)

In [10]:
print(np.stack((r, bondprice),axis=1)[displayrows])

[[ 1.5000e-01 -1.4273e+09]
 [ 1.4000e-01  1.6361e+08]
 [ 1.3000e-01  2.2294e+07]
 [ 1.2000e-01 -1.3724e+06]
 [ 1.1000e-01 -1.3361e+05]
 [ 1.0000e-01  3.2966e+03]
 [ 9.0000e-02  1.3021e+02]
 [ 8.0000e-02  7.7128e-01]
 [ 7.0000e-02  7.7385e-01]
 [ 6.0000e-02  7.7643e-01]
 [ 5.0000e-02  7.7902e-01]
 [ 4.0000e-02  7.8162e-01]
 [ 3.0000e-02  7.8423e-01]
 [ 2.0000e-02  7.8685e-01]
 [ 1.0000e-02  1.4165e+03]
 [-3.3307e-16  5.1498e+04]]


## (d)

\begin{align*}
& \text{Known by Taylor Approximation: } f(x) = f(a) + f'(a)(x-a) + O((x-a)^2) \\
& \text{Suppose } a = x, \text{ and } x = x+h, \text{ then } f(x+h) = f(x) + f'(x)(h) + O(h^2) \Rightarrow f(x+h)-f(x)-f'(x) \times h = \frac{O(h^2)}{h} \\ 
& |\frac{f(x+h)-f(x)}{h} - f'(x)| = O(h) \\ 

& \text{Similarly: } \\
& f(x+h) = f(x) + f'(x)h + \frac{1}{2}f''(x)h^2 + O(h^3), \\
& f(x-h) = f(x) - f'(x)h + \frac{1}{2}f''(x)h^2 - O(h^3); \\
& \Rightarrow f(x+h) - f(x-h) = 2hf'(x) + 2O(h^3) \\ 
& |\frac{f(x+h) - f(x-h)}{2h} - f'(x)| = O(h^2) \\ 
\end{align*}


## (e)

In [11]:
hw3FD_up = FDexplicitEngine(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=True)
(r, bondprice) = hw3FD_up.price_bond_vasicek(hw3contract,hw3dynamics)
np.set_printoptions(precision=4,suppress=True)
displayrows=(r<0.15+hw3FD.deltar/2) & (r>0.0-hw3FD.deltar/2)
print(np.stack((r, bondprice),axis=1)[displayrows])

[[ 0.15    0.7536]
 [ 0.14    0.7561]
 [ 0.13    0.7586]
 [ 0.12    0.7611]
 [ 0.11    0.7637]
 [ 0.1     0.7662]
 [ 0.09    0.7688]
 [ 0.08    0.7713]
 [ 0.07    0.7739]
 [ 0.06    0.7765]
 [ 0.05    0.7791]
 [ 0.04    0.7817]
 [ 0.03    0.7843]
 [ 0.02    0.7869]
 [ 0.01    0.7895]
 [-0.      0.7922]]


In [12]:
hw3FD_cd = FDexplicitEngine(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=False)
(r, bondprice) = hw3FD_cd.price_bond_vasicek(hw3contract,hw3dynamics)
np.set_printoptions(precision=4,suppress=True)
displayrows=(r<0.15+hw3FD_cd.deltar/2) & (r>0.0-hw3FD_cd.deltar/2)
print(np.stack((r, bondprice),1)[displayrows])

[[ 1.5000e-01 -1.4273e+09]
 [ 1.4000e-01  1.6361e+08]
 [ 1.3000e-01  2.2294e+07]
 [ 1.2000e-01 -1.3724e+06]
 [ 1.1000e-01 -1.3361e+05]
 [ 1.0000e-01  3.2966e+03]
 [ 9.0000e-02  1.3021e+02]
 [ 8.0000e-02  7.7128e-01]
 [ 7.0000e-02  7.7385e-01]
 [ 6.0000e-02  7.7643e-01]
 [ 5.0000e-02  7.7902e-01]
 [ 4.0000e-02  7.8162e-01]
 [ 3.0000e-02  7.8423e-01]
 [ 2.0000e-02  7.8685e-01]
 [ 1.0000e-02  1.4165e+03]
 [-3.3307e-16  5.1498e+04]]


The upwind method is more accurate. 

## (f)
less, greater 

## (g)

When $r_0=0.12$, $P_0=0.7611$, so $YTM=log(P_T/P_0)/(T-t)=log(1/0.7611)/5=0.0546$.

When $r_0=0.02$, $P_0= 0.7869$, so $YTM=log(P_T/P_0)/(T-t)=log(1/ 0.7869)/5=0.0479$. 

When $r_0=0.12$, the drift term is negative. This negative drift serves to pull down $r_t$ in subsequent periods. Inversely, when $r_0=0.02$, the drift term is positive. This postive drift serves to moves up $r_t$ in subsequent periods. 

# Problem 2 

## (a)

\begin{align*}

& \Delta = N(d_1) = N(\frac{ln(\frac{S_0}{K})+(r+\frac{\sigma^2}{2})T}{\sigma \sqrt{T}})\\

& N^{-1}(\Delta) = \frac{ln(\frac{S_0}{K})+(r+\frac{\sigma^2}{2})T}{\sigma \sqrt{T}}\\
& N^{-1}(\Delta)\sigma \sqrt{T} - (r+\frac{\sigma^2}{2})T = ln(\frac{S_0}{K})\\

& K = \frac{S_0}{e^{N^{-1}(\Delta)\sigma \sqrt{T} - (r+\frac{\sigma^2}{2})T}}\\
\end{align*}

## (b)

In [13]:
from scipy.stats import norm

In [14]:
def calc_K(sigma,T,Delta,r,S0):
    part1 = norm.ppf(Delta)*sigma*np.sqrt(T) - (r+ (sigma**2/2))*T
    k = S0/np.exp(part1) 
    return k 

def black_scholes_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    return call_price


In [15]:
k25 = calc_K(0.4,1/12,0.25,0.01,300)
k75 = calc_K(0.4,1/12,0.75,0.01,300)

c25 = black_scholes_call_price(300,k25,1/12,0.01,0.4)
c75 = black_scholes_call_price(300,k75,1/12,0.01,0.4)

In [16]:
print(f'The strike of delta-25 is {round(k25,4)}, and the call price is {round(c25,4)}.')
print(f'The strike of delta-75 is {round(k75,4)}, and the call price is {round(c75,4)}.')

The strike of delta-25 is 326.7404, and the call price is 4.8826.
The strike of delta-75 is 279.6109, and the call price is 26.1036.


## (c)

In [17]:
lambda25 = 0.25* 300/c25
lambda75 = 0.75*300/c75
print(f'lambda of delta25: {lambda25}')
print(f'lambda of delta75: {lambda75}')

lambda of delta25: 15.360693494609105
lambda of delta75: 8.6195130132366


Delta-25 gives more leverage. 